In [ ]:
# 최신 YOLOv11 설치
!pip uninstall -y ultralytics
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -e .

In [ ]:
# 라이브러리 설치
!pip install albumentations roboflow opencv-python

In [ ]:
import os
import cv2
from glob import glob
import albumentations as A
from roboflow import Roboflow
from ultralytics import YOLO

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="")
project = rf.workspace("").project("")
dataset = project.version(1).download("yolov11")

print("실제 경로:", dataset.location)


In [ ]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.MotionBlur(p=0.1),
    A.Resize(640, 640)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

In [ ]:
def preprocess_for_yolov11(src_dir, dst_dir):
    os.makedirs(os.path.join(dst_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(dst_dir, "labels"), exist_ok=True)
    image_paths = glob(os.path.join(src_dir, "images", "*.jpg"))

    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is None:
            continue
        filename = os.path.basename(img_path)
        label_path = img_path.replace("images", "labels").replace(".jpg", ".txt")

        bboxes, class_labels = [], []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    cls = int(parts[0])
                    bbox = [float(x) for x in parts[1:]]
                    class_labels.append(cls)
                    bboxes.append(bbox)

        if not bboxes:
            continue

        try:
            transformed = transform(image=img, bboxes=bboxes, class_labels=class_labels)
        except:
            continue

        transformed_image = transformed["image"]
        transformed_bboxes = transformed["bboxes"]
        transformed_labels = transformed["class_labels"]

        cv2.imwrite(os.path.join(dst_dir, "images", filename), transformed_image)

        with open(os.path.join(dst_dir, "labels", filename.replace(".jpg", ".txt")), "w") as f:
            for cls, bbox in zip(transformed_labels, transformed_bboxes):
                f.write(f"{cls} {' '.join(map(str, bbox))}\n")

In [ ]:
# 진짜 이미지가 들어있는 경로로 교체
original_path = "/content/ultralytics/Fire-Smoke-Detection-Yolov11-1"
preprocessed_path = "/content/preprocessed_yolov11"

# 전처리 다시 실행
preprocess_for_yolov11(os.path.join(original_path, "train"),
                       os.path.join(preprocessed_path, "train"))

preprocess_for_yolov11(os.path.join(original_path, "valid"),
                       os.path.join(preprocessed_path, "valid"))


In [ ]:
# data.yaml 수정: 클래스 2개로!
data_yaml = f"""
train: {os.path.join(preprocessed_path, "train/images")}
val: {os.path.join(preprocessed_path, "valid/images")}
nc: 2
names: ["Fire", "Smoke"]
"""

with open(os.path.join(preprocessed_path, "data.yaml"), "w") as f:
    f.write(data_yaml)

print(" data.yaml 생성 완료")

In [ ]:
from ultralytics import YOLO

model = YOLO("/content/yolo11s.pt")

results = model.train(
    data=os.path.join(preprocessed_path, "data.yaml"),
    epochs=100,
    imgsz=640,
    batch=16,
    name="fire_smoke_yolov11s",
    iou=0.5,
    lr0=0.002,
    amp=False,
    workers=2
)